# Web Scraping of "Motorola Mobiles" From Amazon

In [ ]:
#importing necessary libraries

from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd

In [ ]:
#make some functions to get name, description, price and rating from link

def get_name(soup):
    try:
        name= soup.find('span', attrs={'class': 'a-size-medium a-color-base a-text-normal'})
        index = name.text.find('(')
        name = name.text[:index].strip()
    except:
        name = ""
    return name


def get_description(soup):
    try:
        description = soup.find('span', attrs={'class': 'a-size-medium a-color-base a-text-normal'})
        index = description.text.find('(')
        description = description.text[index:].strip()
    except:
        description = ""
    return description
            
def get_price(soup):
    try:
        price = soup.find('span', attrs={'class': 'a-price-whole'})
        actual_price = price.text.string.strip()
    except:
        actual_price = ""
    return actual_price


def get_rate(soup):
    try:
        rating = soup.find('span', attrs={'class': 'a-icon-alt'})
        Rate_text = rating.text.split()[0]
        Rate = Rate_text.string.strip()
    except:
        Rate = ""
    return Rate

In [ ]:
if __name__ == '__main__':
    #Request for header
    HEADERS = ({'User-Agent':"", 'Accept-Language':'en-US, en;q=0.5'})
    
    # Webpage url 
    url = 'https://www.amazon.in/s?k=motorola+mobiles&crid=2N4M4MFOLYPOX&sprefix=%2Caps%2C262&ref=nb_sb_ss_recent_1_0_recent'

    # HTTP request
    webpage = requests.get(url, headers = HEADERS)

    #Soup object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")
    
    #fetch the n numbers of link
    links = soup.find_all('a', attrs={'class': 'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})
    link_list = []
    
    # loop for extracting links from tag objects and put it into link_list
    for lnk in links:
        link_list.append(lnk.get('href'))
    
    amaze_dict = {'Title':[], 'Description':[], 'Price':[], 'Ratings':[]}

    # loop for extracting product details from each link
    for link in link_list:
        webpage_link = requests.get("https://www.amazon.in" + link, headers = HEADERS)
        new_soup = BeautifulSoup(webpage_link.content, "html.parser")
    
        # function calls to display all necessary product information
        amaze_dict['Title'].append(get_name(new_soup))
        amaze_dict['Description'].append(get_description(new_soup))
        amaze_dict['Price'].append(get_price(new_soup))
        amaze_dict['Ratings'].append(get_rate(new_soup))
        

    # make dataframe using pandas and store file as csv
    amazon_df = pd.DataFrame.from_dict(amaze_dict)

    amazon_df['Title'].replace("", np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset = ['Title'])

    amazon_df.to_csv("amazon_data.csv", header=True, index=False)